### Import libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
import random


tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2022-11-17 17:42:58.238064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.10.0


### Set seed for reproducibility

In [2]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Suppress warnings

In [3]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### Models metadata

In [4]:
input_shape = (96, 96, 3)
batch_size = 32
epochs = 200

### Transfer learning

Xception= supernet

In [5]:
dataset_dir = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data'
training_dir = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/training'
validation_dir = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/validation'
test_dir = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/test'

In [6]:
# Images are divided into folders, one for each class.
# If the images are organized in such a way, we can exploit the
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator( preprocessing_function=preprocess_input)
valid_data_gen = ImageDataGenerator( preprocessing_function=preprocess_input)
test_data_gen = ImageDataGenerator( preprocessing_function=preprocess_input)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(96, 96),
                                               color_mode='rgb',
                                               classes=None,  # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed,
                                               )
valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(96, 96),
                                               color_mode='rgb',
                                               classes=None,  # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed,
                                               )
test_gen = train_data_gen.flow_from_directory(directory=test_dir,
                                              target_size=(96, 96),
                                              color_mode='rgb',
                                              classes=None,  # can be set to labels
                                              class_mode='categorical',
                                              batch_size=32,
                                              shuffle=False,
                                              seed=seed,
                                              )

Found 2669 images belonging to 8 classes.
Found 381 images belonging to 8 classes.
Found 492 images belonging to 8 classes.


In [7]:
# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=20,
                                        horizontal_flip= True,
                                        vertical_flip= True,
                                        brightness_range=(0.6,1.4),
                                        zoom_range=0.6,
                                        fill_mode='nearest',
                                         # rescale value is multiplied to the image
                                        preprocessing_function=preprocess_input)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=training_dir,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=32,
                                                       shuffle=True,
                                                       seed=seed,
                                                       )

Found 2669 images belonging to 8 classes.


In [8]:
# Download and plot the Xception model
supernet = tfk.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3)
)
supernet.summary()
tfk.utils.plot_model(supernet)

Metal device set to: AMD Radeon Pro 560X
Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 47, 47, 32)   864         ['input_1[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 47, 47, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                                  
 block1_conv1_act (Activation)  (None, 47, 47, 32)

In [39]:
# Use the supernet as feature extractor
supernet.trainable = False #do not train supernet weights -> in this phase i will update only classifier weights starting from vgg original weights

inputs = tfk.Input(shape=(96,96,3))
x = supernet(inputs)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.2, seed=seed)(x)
x = tfkl.Dense(
    128,
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed))(x)
x = tfkl.Dropout(0.2, seed=seed)(x)
outputs = tfkl.Dense(
    8,
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 Flattening (Flatten)        (None, 18432)             0         
                                                                 
 dropout_8 (Dropout)         (None, 18432)             0         
                                                                 
 dense_12 (Dense)            (None, 128)               2359424   
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 8)                 1032  

In [29]:
# Build the complete model
# puttin first the Xception net
# then a GAP layer, a Dense with LeakyReLU regularization
# and finally the output layer
inputs = tfk.Input(shape=(96,96,3))

x = supernet(inputs)

# Global Average Pooling Layer -----------------------------------------------------------
glob_pooling = tfkl.GlobalAveragePooling2D(name='GlobalPooling')(x)


# Dense Layer -----------------------------------------------------------
x = tfkl.Dense(
    512,
    kernel_initializer = tfk.initializers.GlorotUniform(seed)
)(glob_pooling)

leaky_relu_layer = tfkl.LeakyReLU()(x)

x = tfkl.Dropout(0.3)(leaky_relu_layer)


# Output Layer -----------------------------------------------------------
outputs = tfkl.Dense(
    8,
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed)
)(x)


tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')


tl_model.compile(
    loss=tfk.losses.CategoricalCrossentropy(),
    optimizer=tfk.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy', tf.metrics.Precision(), tf.metrics.Recall()]
)
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 GlobalPooling (GlobalAverag  (None, 2048)             0         
 ePooling2D)                                                     
                                                                 
 dense_10 (Dense)            (None, 512)               1049088   
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                             

In [40]:
# Train the model
tl_history = tl_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

Epoch 1/200
84/84 [==============================] - 22s 232ms/step - loss: 2.0855 - accuracy: 0.2814 - val_loss: 1.7657 - val_accuracy: 0.3438
Epoch 2/200
84/84 [==============================] - 15s 174ms/step - loss: 1.7037 - accuracy: 0.3661 - val_loss: 1.6726 - val_accuracy: 0.3911
Epoch 3/200
84/84 [==============================] - 13s 159ms/step - loss: 1.6601 - accuracy: 0.3586 - val_loss: 1.5749 - val_accuracy: 0.4042
Epoch 4/200
84/84 [==============================] - 13s 156ms/step - loss: 1.6384 - accuracy: 0.3855 - val_loss: 1.5936 - val_accuracy: 0.4147
Epoch 5/200
84/84 [==============================] - 13s 157ms/step - loss: 1.5844 - accuracy: 0.3990 - val_loss: 1.4998 - val_accuracy: 0.4514
Epoch 6/200
84/84 [==============================] - 13s 155ms/step - loss: 1.5636 - accuracy: 0.4170 - val_loss: 1.4884 - val_accuracy: 0.4619
Epoch 7/200
84/84 [==============================] - 13s 156ms/step - loss: 1.5464 - accuracy: 0.4189 - val_loss: 1.4819 - val_accuracy:

In [41]:
# Save the best model
tl_model.save('TransferLearningModel')
del tl_model

In [43]:
model_aug = tfk.models.load_model("TransferLearningModel")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with exception")
print(model_aug_test_metrics)

16/16 [==============================] - 6s 231ms/step - loss: 1.2959 - accuracy: 0.5386

Test metrics with exception
{'loss': 1.2959258556365967, 'accuracy': 0.5386178493499756}
16/16 [==============================] - 5s 209ms/step - loss: 1.2959 - accuracy: 0.5386

Test metrics with exception
{'loss': 1.2959258556365967, 'accuracy': 0.5386178493499756}


### Fine tuning

In [44]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModel')
ft_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 Flattening (Flatten)        (None, 18432)             0         
                                                                 
 dropout_8 (Dropout)         (None, 18432)             0         
                                                                 
 dense_12 (Dense)            (None, 128)               2359424   
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 8)                 1032  

In [45]:
# Set all VGG layers to True
ft_model.get_layer('xception').trainable = True
for i, layer in enumerate(ft_model.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)

0 input_1 True
1 block1_conv1 True
2 block1_conv1_bn True
3 block1_conv1_act True
4 block1_conv2 True
5 block1_conv2_bn True
6 block1_conv2_act True
7 block2_sepconv1 True
8 block2_sepconv1_bn True
9 block2_sepconv2_act True
10 block2_sepconv2 True
11 block2_sepconv2_bn True
12 conv2d True
13 block2_pool True
14 batch_normalization True
15 add True
16 block3_sepconv1_act True
17 block3_sepconv1 True
18 block3_sepconv1_bn True
19 block3_sepconv2_act True
20 block3_sepconv2 True
21 block3_sepconv2_bn True
22 conv2d_1 True
23 block3_pool True
24 batch_normalization_1 True
25 add_1 True
26 block4_sepconv1_act True
27 block4_sepconv1 True
28 block4_sepconv1_bn True
29 block4_sepconv2_act True
30 block4_sepconv2 True
31 block4_sepconv2_bn True
32 conv2d_2 True
33 block4_pool True
34 batch_normalization_2 True
35 add_2 True
36 block5_sepconv1_act True
37 block5_sepconv1 True
38 block5_sepconv1_bn True
39 block5_sepconv2_act True
40 block5_sepconv2 True
41 block5_sepconv2_bn True
42 block5_sep

In [46]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(ft_model.get_layer('xception').layers[:100]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

0 input_1 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d False
13 block2_pool False
14 batch_normalization False
15 add False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_1 False
23 block3_pool False
24 batch_normalization_1 False
25 add_1 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_2 False
33 block4_pool False
34 batch_normalization_2 False
35 add_2 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5_sepconv2 False

In [47]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [48]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

Epoch 1/200
84/84 [==============================] - 29s 291ms/step - loss: 1.5844 - accuracy: 0.4166 - val_loss: 1.6141 - val_accuracy: 0.4777
Epoch 2/200
84/84 [==============================] - 18s 217ms/step - loss: 1.4200 - accuracy: 0.4567 - val_loss: 1.5235 - val_accuracy: 0.4252
Epoch 3/200
84/84 [==============================] - 18s 218ms/step - loss: 1.3626 - accuracy: 0.4938 - val_loss: 1.2899 - val_accuracy: 0.5197
Epoch 4/200
84/84 [==============================] - 19s 219ms/step - loss: 1.3046 - accuracy: 0.5111 - val_loss: 1.2782 - val_accuracy: 0.5328
Epoch 5/200
84/84 [==============================] - 18s 214ms/step - loss: 1.2391 - accuracy: 0.5362 - val_loss: 1.3975 - val_accuracy: 0.4409
Epoch 6/200
84/84 [==============================] - 18s 215ms/step - loss: 1.2210 - accuracy: 0.5463 - val_loss: 1.6779 - val_accuracy: 0.4541
Epoch 7/200
84/84 [==============================] - 18s 218ms/step - loss: 1.1819 - accuracy: 0.5695 - val_loss: 1.4046 - val_accuracy:

In [49]:
ft_model.save('FineTuningModel')
del ft_model

In [50]:
model_aug = tfk.models.load_model("FineTuningModel")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with exception")
print(model_aug_test_metrics)

16/16 [==============================] - 5s 244ms/step - loss: 0.8385 - accuracy: 0.7419

Test metrics with exception
{'loss': 0.8385012745857239, 'accuracy': 0.7418698668479919}
